## Concurrency with asyncio


### Thread versus coroutine: a comparison


In [5]:
# example thread

import threading
import itertools
import sys
import time

class Signal:
    go = True
    
def spin(msg, signal):
    write, flush = sys.stdout.write, sys.stdout.flush
    for char in itertools.cycle('|/-\\'):
        status = char + ' ' + msg
        write(msg)
        flush()
        write('\x08'*len(status))
        time.sleep(.1)
        if not signal.go:
            break
    write(' '*len(status) + '\x08' * len(status))
    
def slow_function():
    time.sleep(5)
    return 42

def supervisor():
    signal = Signal()
    spinner = threading.Thread(target=spin, args=('thinking!', signal))
    print('spinner object: ', spinner)
    spinner.start()
    result = slow_function()
    signal.go = False
    spinner.join()
    return result

def main():
    result = supervisor()
    print(' Answer: ',result)

main()



spinner object:  <Thread(Thread-6, initial)>
thinkingthinkingthinkingthinkingthinkingthinkingthinkingthinkingthinkingthinkingthinkingthinkingthinkingthinkingthinkingthinkingthinkingthinkingthinkingthinkingthinkingthinkingthinkingthinkingthinkingthinkingthinkingthinkingthinkingthinkingthinkingthinkingthinkingthinkingthinkingthinkingthinkingthinkingthinkingthinkingthinkingthinkingthinkingthinkingthinkingthinkingthinkingthinkingthinkingthinking           Answer:  42


In [8]:
# example 
# asyncio

import asyncio
import itertools
import sys

@asyncio.coroutine
def spin(msg):
    write,flush = sys.stdout.write, sys.stdout.flush
    for char in itertools.cycle('|/-\\'):
        status = char + ' ' + msg
        write(status)
        flush()
        write('\x08' * len(status))
        try:
            yield from asyncio.sleep(.1)
        except asyncio.CancelledError:
            break
    write(' '*len(status)+'\x08'*len(status))
    

@asyncio.coroutine
def slow_function():
    yield from asyncio.sleep(3)
    return 42

@asyncio.coroutine
def supervisor():
    spinner = asyncio.async(spin('thinking!'))
    print('spinner object:', spinner)
    result = yield from slow_function()
    spinner.cancel()
    return result

def main():
    loop = asyncio.get_event_loop()
    result = loop.run_until_complete(supervisor())
    loop.close()
    print('Answer: ', result)
    
main()


spinner object: <Task pending coro=<spin() running at <ipython-input-8-2fe6b17db71c>:8>>
| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking          Answer:  42


### asyncio.Future: non-blocking by design


In [31]:
# example 煎蛋
import requests
import aiohttp
import asyncio
import bs4
import datetime

PATH = 'pics/'

@asyncio.coroutine
def download_pic(url):
    resp = yield from aiohttp.requests('GET', url)
    img = yield from resp.read()
    return img

def save(url, content):
    name = url.split('/')[-1]
    with open(PATH + name, 'wb')as f:
        f.write(content)

@asyncio.coroutine      
def download_one(url):
    img = yield from download_pic(url)
    save(url, img)
    return url

def get_pic_urls(url):
    
    req = requests.get(url)
    soup = bs4.BeautifulSoup(req.text, 'lxml')
    soup_pic_urls = soup.find_all('a', class_='view_img_link')
    return [i['href'] for i in soup_pic_urls]


def download_lst():
    url = 'http://jandan.net/ooxx/page-231#comments'
    return get_pic_urls(url)


def download_many():
    lst = download_lst()
    lst = ['https:'+i for i in lst]
    loop = asyncio.get_event_loop()
    to_do = [download_one(u) for u in lst]
    wait_coro = asyncio.wait(to_do)
    res, _ = loop.run_until_complete(wait_coro)
    loop.close()
    
    
    return len(res)

        
def main():
    start = datetime.datetime.now()
    download_many()
    end = datetime.datetime.now()
    print((end-start).seconds)
    
main()

RuntimeError: Event loop is closed